# imports

In [1]:
from bs4 import BeautifulSoup
from datetime import date
from requests import get
import pandas as pd

# crawler

## defining url

In [2]:
url = 'https://www.pararius.com/apartments/nederland/page-'
response = get(url+'1')
html_soup = BeautifulSoup(response.text, 'html.parser')

## get numbers to define iteractions

In [3]:
num_pages = html_soup.find_all('a',{"class":"pagination__link"})[4].text
num_pages = int(num_pages)

num_rents = html_soup.find_all('div',{"class":"pagination__summary"})[0].text
num_rents = int(num_rents.split()[-2])

In [4]:
print(url+'1')
print(num_pages)
print(num_rents)

https://www.pararius.com/apartments/nederland/page-1
184
5508


## get total html soup

In [5]:
houses = []
for count in range(1, num_pages+1):
    print(count)
    try:
        response = get(url + str(count))
        html_soup = BeautifulSoup(response.text, 'html.parser')
        house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
        houses.append(house_data)
    except:
        pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184


## get data

In [6]:
description_=[]
price_=[]
address_=[]
street_=[]
agency_=[]
irl_=[]
image_=[]

for n_pages in range (num_pages):
    n_pages_len = len(houses[n_pages])
    for n_rents in range (n_pages_len):
        num = (houses[n_pages][n_rents])

        try:
            p1_key = num.find_all('span',{"class":"illustrated-features__term"})[0].text
            p1_value = num.find_all('span',{"class":"illustrated-features__description"})[0].text
        except:
            p1_key = None
            p1_value = None
        try:
            p2_key = num.find_all('span',{"class":"illustrated-features__term"})[1].text
            p2_value = num.find_all('span',{"class":"illustrated-features__description"})[1].text
        except:
            p2_key = None
            p2_value = None
        try:
            p3_key = num.find_all('span',{"class":"illustrated-features__term"})[2].text
            p3_value = num.find_all('span',{"class":"illustrated-features__description"})[2].text
        except:
            p3_key = None
            p3_value = None
        try:
            p4_key = num.find_all('span',{"class":"illustrated-features__term"})[3].text
            p4_value = num.find_all('span',{"class":"illustrated-features__description"})[3].text
        except:
            p4_key = None
            p4_value = None
        try:
            p5_value = num.find_all('span',{"class":"listing-status--under-option"})[0].text
        except:
            p5_value = None

        description = {
            p1_key:p1_value,
            p2_key:p2_value,
            p3_key:p3_value,
            p4_key:p4_value,
            'status':p5_value,
        }
        
        description_.append(description)
            
        price = num.find_all('span',{"class":"listing-search-item__price"})[0].text
        price_.append(price)
                
        address = num.find('div',{"class":"listing-search-item__location"}).text
        address_.append(address)

        street = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0].text
        street_.append(street)

        irl = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0]['href']
        irl_.append(irl)
        
        image = num.find_all('img')[0]['src']
        image_.append(image)

        agency = num.find_all('a', href=True)[2].text
        agency_.append(agency)

## format data as pandas dataframe

In [7]:
df_pararius_1 = pd.DataFrame({
    'price':price_,
    'address':address_,
    'street':street_,
    'agency':agency_,
    'irl':irl_,
    'image':image_,
})
df_pararius_2 = pd.DataFrame(description_)

In [8]:
df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)

<ipython-input-8-22791b88bcc6>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)


## formating and cleaning data

In [9]:
from datetime import date
today = date.today()

In [10]:
#df_pararius
df_pararius_2

,NaN,status
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
5870,None,\n Under option\n
5871,None,\n Under option\n
5872,None,\n Under option\n
5873,None,\n Under option\n


In [11]:
df_pararius = df_pararius.dropna(axis=1, how='all')
df_pararius['price'] = df_pararius['price'].str.replace("\D","",regex=True)
df_pararius['address'] = df_pararius['address'].str.replace('\n|new|  ',"",regex=True)
df_pararius['postcode'] = df_pararius['address'].str.replace("\s","",regex=True).str[0:6]
df_pararius['Living area'] = df_pararius['Living area'].str.replace("\D","",regex=True)
df_pararius['garden area'] = df_pararius['garden area'].str.replace("\D","",regex=True)
df_pararius['status'] = df_pararius['status'].str.replace('\n',"",regex=True)
df_pararius['date'] = str(today)

KeyError: 'Living area'

# Save Data

In [ ]:
today_csv = ("df_pararius_"+ str(today)+'.csv')
df_pararius.to_csv(f'data/temp/{today_csv}')

In [ ]:
df_pararius.loc[df_pararius['plot area'].notna()]['plot area']
list(df_pararius.iloc[[18]]['irl'])